# 資料合併-短程

In [11]:
import os
import pandas as pd

def merge_files(file_paths, output_path):
    data_frames = []
    
    # 逐一讀取檔案
    for file_path in file_paths:
        if os.path.exists(file_path):
            print(f"正在讀取檔案：{file_path}")
            df = pd.read_csv(file_path)
            if not df.empty:
                data_frames.append(df)
            else:
                print(f"檔案 {file_path} 為空，跳過。")
        else:
            print(f"檔案 {file_path} 不存在，跳過。")

    # 合併所有資料
    if data_frames:
        merged_data = pd.concat(data_frames, ignore_index=True)
        merged_data.to_csv(output_path, index=False, encoding='utf-8-sig')
        print(f"合併完成，結果儲存至：{output_path}")
    else:
        print("沒有任何資料可供合併。")

# 主程式
base_path = '/Users/yuchingchen/Documents/專題/cleaned_data/data/short'
output_path = '/Users/yuchingchen/Documents/專題/cleaned_data/short_flight.csv'

# 檔案名稱清單
file_paths = [
    f'{base_path}/tokyo.csv',
    f'{base_path}/hongkong.csv',
    f'{base_path}/singapore.csv',
    f'{base_path}/bangkok.csv',
    f'{base_path}/seoul.csv'
]

# 合併檔案
merge_files(file_paths, output_path)

正在讀取檔案：/Users/yuchingchen/Documents/專題/cleaned_data/data/short/tokyo.csv
正在讀取檔案：/Users/yuchingchen/Documents/專題/cleaned_data/data/short/hongkong.csv
正在讀取檔案：/Users/yuchingchen/Documents/專題/cleaned_data/data/short/singapore.csv
正在讀取檔案：/Users/yuchingchen/Documents/專題/cleaned_data/data/short/bangkok.csv
正在讀取檔案：/Users/yuchingchen/Documents/專題/cleaned_data/data/short/seoul.csv
合併完成，結果儲存至：/Users/yuchingchen/Documents/專題/cleaned_data/short_flight.csv


# 短程航班-經濟艙

In [6]:
import pandas as pd
import numpy as np
import re
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns

# --------------------------
# 1. 載入資料及初步處理
# --------------------------
data_path = '/Users/yuchingchen/Documents/專題/cleaned_data/short_flight.csv'
data = pd.read_csv(data_path)

# 統一「出發日期」格式為 "YYYY-MM-DD"
data["出發日期"] = pd.to_datetime(
    data["出發日期"], 
    format="%Y-%m-%d", 
    errors="coerce"
).combine_first(
    pd.to_datetime(data["出發日期"], format="%Y/%m/%d", errors="coerce")
)
data["出發日期"] = data["出發日期"].dt.strftime("%Y-%m-%d")

# --------------------------
# 2. 計算對數變數與新增最低價格天數
# --------------------------
data["平均價格_log"] = data["平均價格"].apply(lambda x: np.log1p(x) if pd.notnull(x) else np.nan)
data["最低價格_log"] = data["最低價格"].apply(lambda x: np.log1p(x) if pd.notnull(x) else np.nan)
data["價格變異_log"] = data["價格變異"].apply(lambda x: np.log1p(x) if pd.notnull(x) else np.nan)
data["中位數價格_log"] = data["中位數價格"].apply(lambda x: np.log1p(x) if pd.notnull(x) else np.nan)

# 新增最低價格天數 (91 - '最低價格剩餘天數')
data['最低價格天數'] = 91 - data['最低價格剩餘天數']

# 移除包含 NaN 的數據（針對關鍵欄位）
data = data.dropna(subset=["平均價格_log", "最低價格_log", "最低價格天數", "價格變異_log", "中位數價格_log"])

# 顯示抵達機場代號種類及數量
print(data['抵達機場代號'].value_counts())

# --------------------------
# 3. 假期判斷
# --------------------------
holidays = {
    "台灣": [("2025-01-25", "2025-02-02"), ("2025-02-28", "2025-03-02")],
    "日本": [("2025-02-11", "2025-02-11"), ("2025-02-22", "2025-02-24"), ("2025-03-20", "2025-03-20")],
    "香港": [("2025-01-29", "2025-01-31")],
    "新加坡": [("2025-01-29", "2025-01-30")],
    "韓國": [("2025-01-28", "2025-01-30"), ("2025-03-01", "2025-03-03")]
}

airport_to_region = {
    "HKG": "香港",
    "NRT": "日本",
    "HND": "日本",
    "BKK": "泰國",  # 泰國沒有假期，不處理
    "ICN": "韓國",
    "GMP": "韓國",
    "SIN": "新加坡"
}

# 將「出發日期」轉回 datetime 格式以利比較
data["出發日期"] = pd.to_datetime(data["出發日期"])

def is_holiday(row):
    # 台灣假期判斷（以出發地為台灣）
    for start, end in holidays["台灣"]:
        if pd.to_datetime(start) <= row["出發日期"] <= pd.to_datetime(end):
            return 1
    # 根據抵達機場對應地區判斷假期
    region = airport_to_region.get(row["抵達機場代號"])
    if region and region in holidays:
        for start, end in holidays[region]:
            if pd.to_datetime(start) <= row["出發日期"] <= pd.to_datetime(end):
                return 1
    return 0

data["假期"] = data.apply(is_holiday, axis=1)

# 將「假期」欄位移至「艙等」欄位之後（若「艙等」存在）
cols = data.columns.tolist()
if '假期' in cols and '艙等' in cols:
    cols.insert(cols.index('艙等') + 1, cols.pop(cols.index('假期')))
data = data[cols]

# --------------------------
# 4. 新增額外的地區經濟資料
# --------------------------
additional_data = {
    "Region": ["香港", "日本", "新加坡", "韓國", "泰國"],
    "Cost of Living Index": [70.8, 46.1, 76.7, 60.1, 34.1],
    "GDP (PPP) per capita (in thousand USD)": [71.627, 51.399, 141.553, 60.046, 23.981],
}
region_df = pd.DataFrame(additional_data)

data["Region"] = data["抵達機場代號"].map(airport_to_region)
data = data.merge(region_df, how="left", on="Region")

# --------------------------
# 5. 新增是否為平日欄位
# --------------------------
weekend_days = ["週六", "週日"]
data["是否為平日"] = data["星期"].apply(lambda x: 1 if x in weekend_days else 0)

# 將「是否為平日」欄位移至「是否過夜」欄位之後
cols = data.columns.tolist()
if '是否過夜' in cols and '是否為平日' in cols:
    idx = cols.index("是否過夜") + 1
    cols.insert(idx, cols.pop(cols.index("是否為平日")))
data = data[cols]

# --------------------------
# 6. 新增機場客運量分類欄位
# --------------------------
super_large_airports = ['HND', 'HKG', 'SIN', 'BKK', 'ICN']
medium_large_airports = ['NRT', 'GMP']

data['機場客運量分類'] = data['抵達機場代號'].apply(
    lambda x: 1 if x in super_large_airports else (2 if x in medium_large_airports else None)
)

# 將「機場客運量分類」移至「抵達機場代號」後
cols = data.columns.tolist()
if '抵達機場代號' in cols and '機場客運量分類' in cols:
    arrival_idx = cols.index('抵達機場代號')
    cols.insert(arrival_idx + 1, cols.pop(cols.index('機場客運量分類')))
data = data[cols]

# --------------------------
# 7. 新增「飛行時間_分鐘」欄位
# --------------------------
def convert_flight_time_to_minutes(time_str):
    """
    將 'x 小時 y 分鐘' 格式的字串轉換為分鐘數
    """
    if isinstance(time_str, str):
        match = re.match(r'(\d+)\s*小時(?:\s*(\d+)\s*分鐘)?', time_str)
        if match:
            hours = int(match.group(1))
            minutes = int(match.group(2)) if match.group(2) else 0
            return hours * 60 + minutes
    return None

data['飛行時間_分鐘'] = data['飛行時間'].apply(convert_flight_time_to_minutes)

# 將「飛行時間_分鐘」移至「飛行時間」欄位之後
cols = data.columns.tolist()
if '飛行時間' in cols and '飛行時間_分鐘' in cols:
    flight_idx = cols.index('飛行時間')
    cols.insert(flight_idx + 1, cols.pop(cols.index('飛行時間_分鐘')))
data = data[cols]

# --------------------------
# 8. 匯出處理後的資料
# --------------------------
output_path = '/Users/yuchingchen/Documents/專題/cleaned_data/short_flight_final.csv'
data.to_csv(output_path, index=False)

抵達機場代號
HKG    2293
NRT    1252
BKK     949
ICN     768
SIN     595
HND     587
GMP     121
Name: count, dtype: int64


# 短程航班-商務艙

In [1]:
import pandas as pd
import numpy as np
import re
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns

# --------------------------
# 1. 載入資料及初步處理
# --------------------------
data_path = '/Users/yuchingchen/Documents/專題/cleaned_data/short_flight_business.csv'
data = pd.read_csv(data_path)

# 統一「出發日期」格式為 "YYYY-MM-DD"
data["出發日期"] = pd.to_datetime(
    data["出發日期"], 
    format="%Y-%m-%d", 
    errors="coerce"
).combine_first(
    pd.to_datetime(data["出發日期"], format="%Y/%m/%d", errors="coerce")
)
data["出發日期"] = data["出發日期"].dt.strftime("%Y-%m-%d")

# --------------------------
# 2. 計算對數變數與新增最低價格天數
# --------------------------
data["平均價格_log"] = data["平均價格"].apply(lambda x: np.log1p(x) if pd.notnull(x) else np.nan)
data["最低價格_log"] = data["最低價格"].apply(lambda x: np.log1p(x) if pd.notnull(x) else np.nan)
data["價格變異_log"] = data["價格變異"].apply(lambda x: np.log1p(x) if pd.notnull(x) else np.nan)
data["中位數價格_log"] = data["中位數價格"].apply(lambda x: np.log1p(x) if pd.notnull(x) else np.nan)

# 新增最低價格天數 (91 - '最低價格剩餘天數')
data['最低價格天數'] = 91 - data['最低價格剩餘天數']

# 移除包含 NaN 的數據（針對關鍵欄位）
data = data.dropna(subset=["平均價格_log", "最低價格_log", "最低價格天數", "價格變異_log", "中位數價格_log"])

# 顯示抵達機場代號種類及數量
print(data['抵達機場代號'].value_counts())

# --------------------------
# 3. 假期判斷
# --------------------------
holidays = {
    "台灣": [("2025-01-25", "2025-02-02"), ("2025-02-28", "2025-03-02")],
    "日本": [("2025-02-11", "2025-02-11"), ("2025-02-22", "2025-02-24"), ("2025-03-20", "2025-03-20")],
    "香港": [("2025-01-29", "2025-01-31")],
    "新加坡": [("2025-01-29", "2025-01-30")],
    "韓國": [("2025-01-28", "2025-01-30"), ("2025-03-01", "2025-03-03")]
}

airport_to_region = {
    "HKG": "香港",
    "NRT": "日本",
    "HND": "日本",
    "BKK": "泰國",  # 泰國沒有假期，不處理
    "ICN": "韓國",
    "GMP": "韓國",
    "SIN": "新加坡"
}

# 將「出發日期」轉回 datetime 格式以利比較
data["出發日期"] = pd.to_datetime(data["出發日期"])

def is_holiday(row):
    # 台灣假期判斷（以出發地為台灣）
    for start, end in holidays["台灣"]:
        if pd.to_datetime(start) <= row["出發日期"] <= pd.to_datetime(end):
            return 1
    # 根據抵達機場對應地區判斷假期
    region = airport_to_region.get(row["抵達機場代號"])
    if region and region in holidays:
        for start, end in holidays[region]:
            if pd.to_datetime(start) <= row["出發日期"] <= pd.to_datetime(end):
                return 1
    return 0

data["假期"] = data.apply(is_holiday, axis=1)

# 將「假期」欄位移至「艙等」欄位之後（若「艙等」存在）
cols = data.columns.tolist()
if '假期' in cols and '艙等' in cols:
    cols.insert(cols.index('艙等') + 1, cols.pop(cols.index('假期')))
data = data[cols]

# --------------------------
# 4. 新增額外的地區經濟資料
# --------------------------
additional_data = {
    "Region": ["香港", "日本", "新加坡", "韓國", "泰國"],
    "Cost of Living Index": [70.8, 46.1, 76.7, 60.1, 34.1],
    "GDP (PPP) per capita (in thousand USD)": [71.627, 51.399, 141.553, 60.046, 23.981],
}
region_df = pd.DataFrame(additional_data)

data["Region"] = data["抵達機場代號"].map(airport_to_region)
data = data.merge(region_df, how="left", on="Region")

# --------------------------
# 5. 新增是否為平日欄位
# --------------------------
weekend_days = ["週六", "週日"]
data["是否為平日"] = data["星期"].apply(lambda x: 1 if x in weekend_days else 0)

# 將「是否為平日」欄位移至「是否過夜」欄位之後
cols = data.columns.tolist()
if '是否過夜' in cols and '是否為平日' in cols:
    idx = cols.index("是否過夜") + 1
    cols.insert(idx, cols.pop(cols.index("是否為平日")))
data = data[cols]

# --------------------------
# 6. 新增機場客運量分類欄位
# --------------------------
super_large_airports = ['HND', 'HKG', 'SIN', 'BKK', 'ICN']
medium_large_airports = ['NRT', 'GMP']

data['機場客運量分類'] = data['抵達機場代號'].apply(
    lambda x: 1 if x in super_large_airports else (2 if x in medium_large_airports else None)
)

# 將「機場客運量分類」移至「抵達機場代號」後
cols = data.columns.tolist()
if '抵達機場代號' in cols and '機場客運量分類' in cols:
    arrival_idx = cols.index('抵達機場代號')
    cols.insert(arrival_idx + 1, cols.pop(cols.index('機場客運量分類')))
data = data[cols]

# --------------------------
# 7. 新增「飛行時間_分鐘」欄位
# --------------------------
def convert_flight_time_to_minutes(time_str):
    """
    將 'x 小時 y 分鐘' 格式的字串轉換為分鐘數
    """
    if isinstance(time_str, str):
        match = re.match(r'(\d+)\s*小時(?:\s*(\d+)\s*分鐘)?', time_str)
        if match:
            hours = int(match.group(1))
            minutes = int(match.group(2)) if match.group(2) else 0
            return hours * 60 + minutes
    return None

data['飛行時間_分鐘'] = data['飛行時間'].apply(convert_flight_time_to_minutes)

# 將「飛行時間_分鐘」移至「飛行時間」欄位之後
cols = data.columns.tolist()
if '飛行時間' in cols and '飛行時間_分鐘' in cols:
    flight_idx = cols.index('飛行時間')
    cols.insert(flight_idx + 1, cols.pop(cols.index('飛行時間_分鐘')))
data = data[cols]

# --------------------------
# 8. 匯出處理後的資料
# --------------------------
output_path = '/Users/yuchingchen/Documents/專題/cleaned_data/short_flight_business_final.csv'
data.to_csv(output_path, index=False)

NameError: name 'pd' is not defined